In [1]:
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from IPython.display import clear_output

import matplotlib as mpl
import matplotlib.pyplot as plt

import data_parser
%matplotlib inline

In [2]:
# CUDA
if torch.cuda.is_available():
    dev = "cuda:0"
    print("CUDA avaiable")
else:  
    dev = "cpu"
    print("CUDA not avaiable")
device = torch.device(dev)

CUDA avaiable


In [3]:
df = data_parser.data_parse()
df

F      R(1,1)    R(1,2)    R(2,1)      R(2,2)  \
4d80d23000 0    0.000000e+00    4.868168  0.026832  0.026832    4.868165   
           1    2.000000e+08   21.286813  0.756865  0.756865   21.289149   
           2    4.000000e+08   29.888101  1.099127  1.099127   29.895878   
           3    6.000000e+08   36.646929  1.389542  1.389542   36.662283   
           4    8.000000e+08   42.441112  1.652974  1.652974   42.462041   
...                      ...         ...       ...       ...         ...   
5d60d23800 496  9.920000e+10  407.116139 -0.125094 -0.125094  406.739462   
           497  9.940000e+10  407.597227 -0.134832 -0.134832  407.219341   
           498  9.960000e+10  408.078004 -0.144564 -0.144564  407.699021   
           499  9.980000e+10  408.558477 -0.154289 -0.154289  408.178498   
           500  1.000000e+11  409.038650 -0.164001 -0.164001  408.657769   

                      L(1,1)        L(1,2)        L(2,1)        L(2,2)  \
4d80d23000 0    5.310122e-07  2.242849e-08  2.372460e-08  2.958032e-07   
           1    2.963599e-07  1.398684e-08  1.398684e-08  2.963564e-07   
           2    2.916231e-07  1.381295e-08  1.381295e-08  2.916190e-07   
           3    2.894990e-07  1.373125e-08  1.373125e-08  2.894934e-07   
           4    2.882139e-07  1.367905e-08  1.367905e-08  2.882067e-07   
...                      ...           ...           ...           ...   
5d60d23800 496  2.595269e-07  1.151338e-08  1.151338e-08  2.595276e-07   
           497  2.595262e-07  1.151339e-08  1.151339e-08  2.595269e-07   
           498  2.595256e-07  1.151340e-08  1.151340e-08  2.595262e-07   
           499  2.595249e-07  1.151341e-08  1.151341e-08  2.595256e-07   
           500  2.595242e-07  1.151342e-08  1.151342e-08  2.595249e-07   

                      C(1,1)  ...   SI(4,1)   SR(4,2)   SI(4,2)   SR(4,3)  \
4d80d23000 0    1.424892e-10  ...  0.000000  0.996300  0.000000  0.000042   
           1    1.384956e-10  ...  0.001350  0.800984 -0.566244  0.016042   
           2    1.380937e-10  ... -0.002366  0.335738 -0.908265  0.040671   
           3    1.377709e-10  ... -0.003923 -0.229447 -0.931511  0.043128   
           4    1.375511e-10  ... -0.000749 -0.708457 -0.639615  0.021617   
...                      ...  ...       ...       ...       ...       ...   
5d60d23800 496  1.454663e-10  ...  0.005037  0.025167  0.179031  0.022607   
           497  1.454649e-10  ...  0.006172  0.139196  0.114635  0.021826   
           498  1.454636e-10  ...  0.004107  0.179578 -0.009650  0.021493   
           499  1.454623e-10  ... -0.000127  0.125386 -0.128133  0.022210   
           500  1.454610e-10  ... -0.004307  0.005800 -0.178627  0.022659   

                 SI(4,3)   SR(4,4)   SI(4,4)    W  Trap  Length  
4d80d23000 0    0.000000  0.003692  0.000000  4.8   0.2  3000.0  
           1    0.021649 -0.014187 -0.043728  4.8   0.2  3000.0  
           2    0.014571 -0.066581 -0.040258  4.8   0.2  3000.0  
           3   -0.010059 -0.081401  0.006525  4.8   0.2  3000.0  
           4   -0.021466 -0.044871  0.034905  4.8   0.2  3000.0  
...                  ...       ...       ...  ...   ...     ...  
5d60d23800 496 -0.000259 -0.086717  0.001857  5.6   0.2  3800.0  
           497 -0.000528 -0.083515  0.003697  5.6   0.2  3800.0  
           498  0.000224 -0.081418  0.000682  5.6   0.2  3800.0  
           499  0.000617 -0.084224 -0.001650  5.6   0.2  3800.0  
           500 -0.000058 -0.086772  0.000931  5.6   0.2  3800.0  

[220941 rows x 52 columns]

In [6]:
df.xs(0, level=1)

,F,"R(1,1)","R(1,2)","R(2,1)","R(2,2)","L(1,1)","L(1,2)","L(2,1)","L(2,2)","C(1,1)",...,"SI(4,1)","SR(4,2)","SI(4,2)","SR(4,3)","SI(4,3)","SR(4,4)","SI(4,4)",W,Trap,Length
4d80d23000,8.400000e+09,134.649573,4.720956,4.720956,134.420533,2.822641e-07,1.341203e-08,1.341203e-08,2.822581e-07,1.360969e-10,...,0.003511,0.795261,0.190803,0.009589,-0.006920,-0.018943,0.013801,4.8,0.2,3000.0
4d80d23600,8.400000e+09,134.649573,4.720956,4.720956,134.420533,2.822641e-07,1.341203e-08,1.341203e-08,2.822581e-07,1.360969e-10,...,0.004295,0.025437,0.782756,0.038226,-0.001212,-0.075217,0.001214,4.8,0.2,3600.0
4d20d24600,8.400000e+09,146.647008,5.468162,5.468162,146.705868,3.007658e-07,1.513855e-08,1.513854e-08,3.007682e-07,1.277923e-10,...,0.003956,0.648309,-0.340643,0.017679,0.011223,-0.010441,-0.007098,4.2,0.2,4600.0
4d80d23400,8.400000e+09,134.649573,4.720956,4.720956,134.420533,2.822641e-07,1.341203e-08,1.341203e-08,2.822581e-07,1.360969e-10,...,-0.004063,-0.795108,-0.045124,0.008878,-0.001381,-0.017299,0.003004,4.8,0.2,3400.0
602400,8.400000e+09,112.646770,3.541780,3.541780,112.556764,2.474661e-07,1.101182e-08,1.101182e-08,2.474650e-07,1.550951e-10,...,-0.003376,0.402749,-0.737548,0.030798,0.012402,-0.154430,-0.068271,6.0,0.0,2400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4d202000,8.400000e+09,141.741846,5.401107,5.401107,141.737391,2.948734e-07,1.546474e-08,1.546473e-08,2.948726e-07,1.303508e-10,...,-0.000574,-0.547907,0.679945,0.030682,0.019586,-0.028743,-0.019128,4.2,0.0,2000.0
5d402600,8.400000e+09,120.409693,4.033844,4.033844,120.480547,2.614174e-07,1.225952e-08,1.225952e-08,2.614189e-07,1.468749e-10,...,-0.002537,-0.766613,-0.343620,0.012255,-0.011876,-0.044774,0.044292,5.4,0.0,2600.0
5d402200,8.400000e+09,120.409693,4.033844,4.033844,120.480547,2.614174e-07,1.225952e-08,1.225952e-08,2.614189e-07,1.468749e-10,...,0.002311,0.710592,0.486268,0.016895,-0.015811,-0.061767,0.058315,5.4,0.0,2200.0
60d24400,8.400000e+09,115.452372,3.540128,3.540128,115.506348,2.515425e-07,1.072204e-08,1.072204e-08,2.515413e-07,1.525748e-10,...,0.006755,0.271169,0.686935,0.028890,-0.008057,-0.145429,0.038835,6.0,0.2,4400.0


In [10]:
input_cols = ["W", "Trap", "Length"]
frequency_point = 42
output_cols = "SR(1,1)"


X = df.xs(frequency_point, level=1)[input_cols].to_numpy()
# X = np.tile(X, (1, 20))
y = df.xs(frequency_point, level=1)[output_cols].to_numpy().reshape(-1, 1)

array([[-0.01894304],
       [-0.07519653],
       [-0.01043991],
       [-0.01730755],
       [-0.15444031],
       [-0.08200138],
       [-0.07551407],
       [-0.0230248 ],
       [-0.01944868],
       [-0.01481032],
       [-0.05885775],
       [-0.0080565 ],
       [-0.06628055],
       [-0.17185611],
       [-0.15695045],
       [-0.14259505],
       [-0.13824182],
       [-0.05860645],
       [-0.17335555],
       [-0.03165234],
       [-0.09880132],
       [-0.12931791],
       [-0.02095823],
       [-0.01497849],
       [-0.01521461],
       [-0.01304515],
       [-0.01354198],
       [-0.01837691],
       [-0.05677201],
       [-0.0594225 ],
       [-0.1108357 ],
       [-0.02314947],
       [-0.10599847],
       [-0.02420284],
       [-0.02661791],
       [-0.02072672],
       [-0.00863534],
       [-0.17163626],
       [-0.07579604],
       [-0.04054929],
       [-0.00805887],
       [-0.10635197],
       [-0.02712717],
       [-0.01934499],
       [-0.05810818],
       [-0

In [ ]:
class data_processor():
    def __init__(self):
        self.dict = {}
    
    def normalize(self, X, y):
        self.dict['X_std'] = X.std()
        self.dict['X_mean'] = X.mean()
        self.dict['y_std'] = y.std()
        self.dict['y_mean'] = y.mean()
        new_X = (X - self.dict['X_mean']) / self.dict['X_std']
        new_y = (y - self.dict['y_mean']) / self.dict['y_std']
        return new_X, new_y
        
    def denormalize(self, X, y):
        original_X = X * self.dict['X_std'] + self.dict['X_mean']
        original_y = y * self.dict['y_std'] + self.dict['y_mean']
        return original_X, original_y
        